In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/kaggle.json


In [21]:
# Import necessary libraries
import os

# Ensure kaggle.json is in the right location
os.makedirs('/root/.kaggle', exist_ok=True)
!cp /kaggle/input/dataset/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Use Kaggle's API to download the Titanic competition data
!kaggle competitions download -c titanic

# Unzip the downloaded data
!unzip -o titanic.zip -d /kaggle/working/titanic

# Check the contents of the unzipped data
!ls /kaggle/working/titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  titanic.zip
  inflating: /kaggle/working/titanic/gender_submission.csv  
  inflating: /kaggle/working/titanic/test.csv  
  inflating: /kaggle/working/titanic/train.csv  
gender_submission.csv  test.csv  train.csv


In [22]:
import pandas as pd
import numpy as np

train_csv = pd.read_csv('/kaggle/working/titanic/train.csv')
test_csv = pd.read_csv('/kaggle/working/titanic/test.csv')

In [23]:
train_csv['Title'] = train_csv['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

test_csv['Title'] = test_csv['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Define a mapping for title grouping
title_mapping = {
    "Mr": "Mr",
    "Miss": "Miss",
    "Mrs": "Mrs",
    "Master": "Master",
    "Dr": "Professional",
    "Rev": "Professional",
    "Mlle": "Miss",
    "Major": "Military",
    "Col": "Military",
    "Countess": "Nobility",
    "Capt": "Military",
    "Ms": "Miss",
    "Sir": "Nobility",
    "Lady": "Nobility",
    "Mme": "Mrs",
    "Don": "Nobility",
    "Jonkheer": "Nobility"
}

# Apply the mapping to the title column
train_csv['Title'] = train_csv['Title'].map(title_mapping)
test_csv['Title'] = test_csv['Title'].map(title_mapping)

In [24]:
train_csv['Family_Size']=train_csv['SibSp']+train_csv['Parch']
test_csv['Family_Size']=test_csv['SibSp']+test_csv['Parch']

train_csv['Age*Class']=train_csv['Age']*train_csv['Pclass']
test_csv['Age*Class']=test_csv['Age']*test_csv['Pclass']

train_csv['Fare_Per_Person']=train_csv['Fare']/(train_csv['Family_Size']+1)
test_csv['Fare_Per_Person']=test_csv['Fare']/(test_csv['Family_Size']+1)

# For the train dataset
train_csv['Cabin'] = train_csv['Cabin'].fillna('Unknown')
train_csv['Deck'] = train_csv['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'Unknown')

# For the test dataset
test_csv['Cabin'] = test_csv['Cabin'].fillna('Unknown')
test_csv['Deck'] = test_csv['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'Unknown')

train_csv.drop('Ticket', axis=1, inplace=True)
test_csv.drop('Ticket', axis=1, inplace=True)

train_csv.drop('Name', axis=1, inplace=True)
test_csv.drop('Name', axis=1, inplace=True)

train_csv.drop('Cabin', axis=1, inplace=True)
test_csv.drop('Cabin', axis=1, inplace=True)

train_csv.drop('PassengerId', axis=1, inplace=True)
test_csv.drop('PassengerId', axis=1, inplace=True)

In [25]:
train_csv.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size,Age*Class,Fare_Per_Person,Deck
0,0,3,male,22.0,1,0,7.2500,S,Mr,1,66.0,3.62500,Unknown
1,1,1,female,38.0,1,0,71.2833,C,Mrs,1,38.0,35.64165,C
2,1,3,female,26.0,0,0,7.9250,S,Miss,0,78.0,7.92500,Unknown
3,1,1,female,35.0,1,0,53.1000,S,Mrs,1,35.0,26.55000,C
4,0,3,male,35.0,0,0,8.0500,S,Mr,0,105.0,8.05000,Unknown


In [26]:
train_csv.dtypes

Survived             int64
Pclass               int64
Sex                 object
Age                float64
SibSp                int64
Parch                int64
Fare               float64
Embarked            object
Title               object
Family_Size          int64
Age*Class          float64
Fare_Per_Person    float64
Deck                object
dtype: object

In [27]:
categorical_columns = ['Sex', 'Embarked', 'Title', 'Deck']
numerical_columns = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Family_Size', 'Age*Class', 'Fare_Per_Person']

In [28]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

def process_categorical_variables(df, categorical_columns, missing_values_strategy='most_frequent'):
    """
    Process categorical variables in a DataFrame.
    
    Args:
    df (pd.DataFrame): The DataFrame to process.
    categorical_columns (list of str): List of column names in df that are categorical.
    missing_values_strategy (str): Strategy for imputing missing values in categorical columns.
                                   Options: 'most_frequent', 'constant' (replaces with 'Unknown')
    
    Returns:
    pd.DataFrame: A DataFrame with processed categorical variables.
    """
    df_processed = df.copy()
    
    # Impute missing values in categorical columns
    if missing_values_strategy == 'constant':
        filler = 'Unknown'
    else:  # Default to 'most_frequent'
        filler = 'most_frequent'
    
    cat_imputer = SimpleImputer(strategy=missing_values_strategy, fill_value=filler)
    df_processed[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])
    
    # Apply one-hot encoding
    one_hot = OneHotEncoder(handle_unknown='ignore', sparse=False)
    encoded_vars = pd.DataFrame(one_hot.fit_transform(df_processed[categorical_columns]))
    
    # One-hot encoding removes index; put it back
    encoded_vars.index = df_processed.index
    
    # Add column names to encoded vars
    encoded_vars.columns = one_hot.get_feature_names_out(categorical_columns)
    
    # Drop original categorical columns and concatenate encoded variables
    df_processed = pd.concat([df_processed.drop(categorical_columns, axis=1), encoded_vars], axis=1)
    
    return df_processed

# Example usage:
# df_processed = process_categorical_variables(df, ['cat_col1', 'cat_col2'], missing_values_strategy='constant')

In [29]:
process_categorical_variables(train_csv, categorical_columns, 'constant')

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Sex_female,...,Title_Professional,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_Unknown
0,0,3,22.0,1,0,7.2500,1,66.0,3.62500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,1,38.0,35.64165,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,0,78.0,7.92500,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,1,35.0,26.55000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3,35.0,0,0,8.0500,0,105.0,8.05000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,54.0,13.00000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1,1,19.0,0,0,30.0000,0,19.0,30.00000,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0,3,NaN,1,2,23.4500,3,NaN,5.86250,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1,1,26.0,0,0,30.0000,0,26.0,30.00000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def process_numerical_variables(df, numerical_columns, missing_values_strategy='mean', scale_data=False, scaling_method='standard'):
    """
    Process numerical variables in a DataFrame.
    
    Args:
    df (pd.DataFrame): The DataFrame to process.
    numerical_columns (list of str): List of column names in df that are numerical.
    missing_values_strategy (str): Strategy for imputing missing values in numerical columns.
                                   Options: 'mean', 'median', 'constant'.
    scale_data (bool): Whether to scale numerical data.
    scaling_method (str): Scaling method to use if scale_data is True.
                          Options: 'standard' for StandardScaler, 'minmax' for MinMaxScaler.
    
    Returns:
    pd.DataFrame: A DataFrame with processed numerical variables.
    """
    df_processed = df.copy()
    
    # Impute missing values in numerical columns
    if missing_values_strategy not in ['mean', 'median', 'constant']:
        raise ValueError("Invalid missing_values_strategy. Choose 'mean', 'median', or 'constant'.")
    
    if missing_values_strategy =='constant':
        num_imputer = SimpleImputer(strategy=missing_values_strategy,  fill_value=-1)
        df_processed[numerical_columns] = num_imputer.fit_transform(df[numerical_columns])
    
    else:
        num_imputer = SimpleImputer(strategy=missing_values_strategy)
        df_processed[numerical_columns] = num_imputer.fit_transform(df[numerical_columns])

    # Scale data if required
    if scale_data:
        if scaling_method == 'standard':
            scaler = StandardScaler()
        elif scaling_method == 'minmax':
            scaler = MinMaxScaler()
        else:
            raise ValueError("Invalid scaling_method. Choose 'standard' or 'minmax'.")
        
        df_processed[numerical_columns] = scaler.fit_transform(df_processed[numerical_columns])

    return df_processed

# Example usage:
# df_processed = process_numerical_variables(df, ['num_col1', 'num_col2'], missing_values_strategy='median', scale_data=True, scaling_method='minmax')

In [31]:
train_df_cat = process_categorical_variables(train_csv, categorical_columns, 'constant')
train_df_processed = process_numerical_variables(train_df_cat, numerical_columns, missing_values_strategy='constant')


test_df_cat = process_categorical_variables(test_csv, categorical_columns, 'constant')
test_df_processed = process_numerical_variables(test_df_cat, numerical_columns, missing_values_strategy='constant')

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [32]:
train_df_processed.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Sex_female,...,Title_Professional,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_Unknown
0,0,3.0,22.0,1.0,0.0,7.2500,1.0,66.0,3.62500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1.0,38.0,1.0,0.0,71.2833,1.0,38.0,35.64165,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3.0,26.0,0.0,0.0,7.9250,0.0,78.0,7.92500,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1.0,35.0,1.0,0.0,53.1000,1.0,35.0,26.55000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3.0,35.0,0.0,0.0,8.0500,0.0,105.0,8.05000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [33]:
from sklearn.model_selection import train_test_split

# Separate target variable
X = train_df_processed.drop('Survived', axis=1)
y = train_df_processed['Survived']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Random Forest Classifier
# n_estimators is the number of trees in the forest
# max_depth is the maximum depth of the trees
# random_state is used for reproducibility of your results
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# Fit the model on the training data
rf_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8268156424581006
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       105
           1       0.79      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179

Confusion Matrix:
[[90 15]
 [16 58]]


# **1. Grid Search**

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],  # Number of trees in the forest
    'max_depth': [None, 10],  # Maximum depth of the tree
    'min_samples_split': [2, 5],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}


grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), 
                           param_grid=param_grid, 
                           cv=5,  # Number of cross-validation folds
                           n_jobs=-1,  # Use all CPU cores
                           verbose=2,  # Higher number gives more information about the process
                           scoring='accuracy'  # Can choose other metrics like 'f1', 'precision', 'recall'
                           )

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [None, 10],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=2)

In [36]:
# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

# Get the best estimator
best_model = grid_search.best_estimator_

Best parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation score: 0.8370530877573131


In [37]:
# Predict on the test data using the best model
y_pred = best_model.predict(X_test)

# Calculate the accuracy or other performance metrics on the test set
test_accuracy = accuracy_score(y_test, y_pred)
test_classification_report = classification_report(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

# Output the performance
print(f"Test Set Accuracy: {test_accuracy}")
print("Test Set Classification Report:")
print(test_classification_report)
print("Test Set Confusion Matrix:")
print(test_confusion_matrix)

Test Set Accuracy: 0.8435754189944135
Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       105
           1       0.85      0.76      0.80        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.84       179
weighted avg       0.84      0.84      0.84       179

Test Set Confusion Matrix:
[[95 10]
 [18 56]]


# **2. Random Search**

In [38]:
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(100, 500),  # Uniformly distributed integer random values within the given range
    'max_depth': [None] + list(range(5, 51, 5)),  # None plus a range from 5 to 50 in steps of 5
    'min_samples_split': randint(2, 20),  # Same as 'n_estimators'
    'min_samples_leaf': randint(1, 10),
    'bootstrap': [True, False],  # Discrete, non-numerical parameters still need to be a list of options
    'max_features': uniform(0.1, 0.9)  # Float values within a range, assuming max_features is used as a float
}

random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42), 
                                   param_distributions=param_dist, 
                                   n_iter=50,  # Number of parameter settings that are sampled
                                   cv=5,  # Number of cross-validation folds
                                   n_jobs=-1,  # Use all CPU cores
                                   verbose=0,  # Higher number gives more information about the process
                                   random_state=42,  # For reproducible results
                                   scoring='accuracy'  # Can choose other metrics like 'f1', 'precision', 'recall'
                                   )

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'bootstrap': False, 'max_depth': 40, 'max_features': 0.14180537144799796, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 120}
Best cross-validation score: 0.8356446370530879


In [39]:
# Get the best parameters and the best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

# Get the best estimator
best_model = random_search.best_estimator_

Best parameters: {'bootstrap': False, 'max_depth': 40, 'max_features': 0.14180537144799796, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 120}
Best cross-validation score: 0.8356446370530879


In [40]:
# Predict on the test data using the best model
y_pred = best_model.predict(X_test)

# Calculate the accuracy or other performance metrics on the test set
test_accuracy = accuracy_score(y_test, y_pred)
test_classification_report = classification_report(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

# Output the performance
print(f"Test Set Accuracy: {test_accuracy}")
print("Test Set Classification Report:")
print(test_classification_report)
print("Test Set Confusion Matrix:")
print(test_confusion_matrix)

Test Set Accuracy: 0.8379888268156425
Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       105
           1       0.84      0.76      0.79        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179

Test Set Confusion Matrix:
[[94 11]
 [18 56]]


# **3. Bayesian Search**

In [44]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import logging

# Set the logging level to WARN to reduce output
# This will display only warnings and errors
logging.getLogger('optuna').setLevel(logging.WARNING)

def objective(trial):
    # Hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    
    # Initialize the classifier with current hyperparameters
    clf = RandomForestClassifier(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_samples_split=min_samples_split, 
        min_samples_leaf=min_samples_leaf, 
        bootstrap=bootstrap,
        random_state=42
    )
    
    # Perform cross-validation
    score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
best_params = study.best_trial.params
print("Best hyperparameters:", best_params)

# Train the best model
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy}")


Best hyperparameters: {'n_estimators': 254, 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 6, 'bootstrap': False}
Test Set Accuracy: 0.8547486033519553


In [49]:
X_final = test_df_processed

# Assuming X_train_encoded is the training data after one-hot encoding
# and X_final is the new data after one-hot encoding

# Get the list of columns from the training set
train_cols = X_train.columns

# Identify the missing columns in the test set and fill them with 0s
for col in train_cols:
    if col not in X_final.columns:
        X_final[col] = 0

# Reorder the columns of X_final to match the order of X_train columns
X_final_aligned = X_final[train_cols]

# Now X_final_aligned has the same column structure as X_train_encoded


best_model.predict(X_final_aligned)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [51]:
X_final_init = pd.read_csv('/kaggle/working/titanic/test.csv')

X_final_init.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [54]:
output = X_final_init[['PassengerId']]

output['Survived'] = best_model.predict(X_final_aligned)

/tmp/ipykernel_42/1711658204.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['Survived'] = best_model.predict(X_final_aligned)


In [57]:
output.to_csv('/kaggle/working/submission.csv', index=False)